In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#importing the data file and displaying the data types
df = pd.read_csv('D1.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19663 entries, 0 to 19662
Data columns (total 8 columns):
InvoiceNo      19663 non-null int64
StockCode      19663 non-null object
Description    19075 non-null object
Quantity       19663 non-null int64
InvoiceDate    19663 non-null object
UnitPrice      19663 non-null float64
CustomerID     15678 non-null float64
Country        19663 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 1.2+ MB


In [3]:
# cleaning data
g = df.groupby('Description')['StockCode'].unique()
print(g)
print()

df2 = df
df2.replace({'85123a':'85123A'}, inplace=True)

g2 = df2.groupby('Description')['StockCode'].unique()
print(g2)
print()
count = df2['StockCode'].count()
print(count)

Description
ASSORTED COLOUR BIRD ORNAMENT                  [84879]
HEART OF WICKER SMALL                          [22469]
JUMBO BAG PINK POLKADOT                        [22386]
JUMBO BAG RED RETROSPOT                       [85099B]
LUNCH BAG  BLACK SKULL.                        [20727]
LUNCH BAG RED RETROSPOT                        [20725]
NATURAL SLATE HEART CHALKBOARD                 [22457]
PACK OF 72 RETROSPOT CAKE CASES                [21212]
PARTY BUNTING                                  [47566]
REGENCY CAKESTAND 3 TIER                       [22423]
SET OF 3 CAKE TINS PANTRY DESIGN               [22720]
WHITE HANGING HEART T-LIGHT HOLDER    [85123A, 85123a]
Name: StockCode, dtype: object

Description
ASSORTED COLOUR BIRD ORNAMENT          [84879]
HEART OF WICKER SMALL                  [22469]
JUMBO BAG PINK POLKADOT                [22386]
JUMBO BAG RED RETROSPOT               [85099B]
LUNCH BAG  BLACK SKULL.                [20727]
LUNCH BAG RED RETROSPOT                [20725]
NA

In [4]:
#drop rows where Description is null
s = df['StockCode'].nunique(dropna = False)
print("StockCode unique = ", s)

d = df['Description'].nunique(dropna = False)
print("Description unique = ", d)

print()
df = df[(df['Description'].notnull())]
df.info()
print()

s = df['StockCode'].nunique(dropna = False)
print("StockCode unique = ", s)

d = df['Description'].nunique(dropna = False)
print("Description unique = ", d)

StockCode unique =  435
Description unique =  13

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19075 entries, 0 to 19662
Data columns (total 8 columns):
InvoiceNo      19075 non-null int64
StockCode      19075 non-null object
Description    19075 non-null object
Quantity       19075 non-null int64
InvoiceDate    19075 non-null object
UnitPrice      19075 non-null float64
CustomerID     15678 non-null float64
Country        19075 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 1.3+ MB

StockCode unique =  12
Description unique =  12


In [5]:
#fix value types
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

df[['StockCode', 'Description', 'Country']] = df[['StockCode', 'Description', 'Country']].astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19075 entries, 0 to 19662
Data columns (total 8 columns):
InvoiceNo      19075 non-null int64
StockCode      19075 non-null category
Description    19075 non-null category
Quantity       19075 non-null int64
InvoiceDate    19075 non-null datetime64[ns]
UnitPrice      19075 non-null float64
CustomerID     15678 non-null float64
Country        19075 non-null category
dtypes: category(3), datetime64[ns](1), float64(2), int64(2)
memory usage: 952.4 KB


N:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
N:\Programs\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [6]:
#drop rows where customer ID is null
df2 = df[(df['CustomerID'].notnull())]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15678 entries, 0 to 19662
Data columns (total 8 columns):
InvoiceNo      15678 non-null int64
StockCode      15678 non-null category
Description    15678 non-null category
Quantity       15678 non-null int64
InvoiceDate    15678 non-null datetime64[ns]
UnitPrice      15678 non-null float64
CustomerID     15678 non-null float64
Country        15678 non-null category
dtypes: category(3), datetime64[ns](1), float64(2), int64(2)
memory usage: 783.2 KB


In [7]:
#compare unique customers and transactions
c = df['CustomerID'].nunique(dropna = False)
print("CustomerID unique = ", c)

i = df['InvoiceNo'].nunique(dropna = False)
print("InvoiceNo unique = ", i)

CustomerID unique =  2882
InvoiceNo unique =  9568


In [8]:
#count of items per transactions
g = df.groupby('InvoiceNo')['StockCode'].nunique().sort_values(ascending = False).value_counts()
print(g)

1     4922
2     2420
3     1086
4      570
5      293
6      128
7       76
8       40
9       24
10       8
11       1
Name: StockCode, dtype: int64


In [12]:
from apyori import apriori

# group by invoices, then list all purchases
transactions1 = df.groupby(['InvoiceNo'])['StockCode'].apply(list)

# type cast the transactions from pandas into normal list format and run apriori
transaction_list1 = list(transactions1)
results1 = list(apriori(transaction_list1, min_confidence = 0.3, min_support=0.04))

In [13]:
def convert_apriori_results_to_pandas_df(results1):
    rules = []

    for rule_set in results1:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add), rule_set.support, rule.confidence, rule.lift])

    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift'])

result_df = convert_apriori_results_to_pandas_df(results1)

# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(10))

#DROPPED NOTHING
#INVOICE ASSOCIATIONS

  Left_side Right_side   Support  Confidence      Lift
3     22386     85099B  0.086225    0.677340  3.097891
4    85099B      22386  0.086225    0.394359  3.097891
0     20725      20727  0.066994    0.409847  3.080449
1     20727      20725  0.066994    0.503535  3.080449
2     20725     85099B  0.060514    0.370205  1.693173
5     22457     85123A  0.043374    0.332266  1.406690
6     22469     85123A  0.040865    0.325562  1.378309


In [22]:
# Q3
from apyori import apriori

# group by invoices, then list all purchases
transactions2 = df2.groupby(['InvoiceNo'])['StockCode'].apply(list)

# type cast the transactions from pandas into normal list format and run apriori
transaction_list2 = list(transactions2)
results2 = list(apriori(transaction_list2, min_confidence = 0.01, min_support=0.01))

In [23]:
def convert_apriori_results_to_pandas_df(results2):
    rules = []

    for rule_set in results2:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add), rule_set.support, rule.confidence, rule.lift])

    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift'])

result_df = convert_apriori_results_to_pandas_df(results2)

# sort all acquired rules descending by lift
result_df = result_df[(result_df['Left_side'] == '20725')].sort_values(by='Support', ascending=False)
print(result_df.head())

#ITEMS WITH 20725
#DROPPED CUSTOMER = NULL
#CUSTOMER ASSOCIATIONS

   Left_side Right_side   Support  Confidence      Lift
13     20725      20727  0.059248    0.401398  3.329463
40     20725     85099B  0.048705    0.329969  1.799568
19     20725      22386  0.027275    0.184783  1.851220
43     20725     85123A  0.025900    0.175466  0.776821
16     20725      21212  0.022576    0.152950  1.297031


In [14]:
#Q4
#count of items per transactions
g = df2.groupby('CustomerID')['StockCode'].nunique().sort_values(ascending = False).value_counts()
print(g)

1     1079
2      664
3      414
4      287
5      176
6      110
7       66
8       38
9       30
10       9
11       4
12       4
Name: StockCode, dtype: int64


In [17]:
# sequence analysis
transactions = df2.groupby(['CustomerID'])['StockCode'].apply(list)
sequences = transactions.values.tolist()

# show the first 10 sequences
print(sequences[:10])

[['22423', '22423', '22423', '22423'], ['22720', '22423'], ['22423', '22720', '22423', '22720'], ['20725'], ['22423'], ['22423', '21212', '22423', '22423'], ['22423'], ['22720', '22423', '47566', '22423', '22423', '22720', '85099B', '22423', '22423'], ['20725'], ['20725', '47566', '22423', '85099B', '20725', '85099B']]


In [27]:
from collections import defaultdict
import subprocess
import re

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    supp_param = '{}%'.format(int(min_sup * 100))
    conf_param = '{}%'.format(int(min_conf * 100))
    subprocess.call(['java', '-jar', 'spmf.jar', 'run', 'RuleGrowth', 
                     'seq_rule_input.txt', 'seq_rule_output.txt', 
                     supp_param, conf_param], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])




In [30]:
get_association_rules(sequences, 0.04, 0.4).sort_values(by='Support', ascending=False)

,Left_rule,Right_rule,Support,Confidence
5,[20725],[20727],0.088164,0.477444
7,[20727],[20725],0.083652,0.526201
0,[20725],[85099B],0.078792,0.426692
10,[22386],[85099B],0.078792,0.610215
9,[22386],[20725],0.053801,0.416667
1,"[20725, 20727]",[85099B],0.047553,0.456667
3,"[85099B, 20727]",[20725],0.046859,0.630841
6,"[20725, 85099B]",[20727],0.046859,0.496324
4,"[85099B, 22386]",[20725],0.045470,0.459649
8,"[20725, 85099B]",[22386],0.044776,0.474265
